In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
%matplotlib qt
import matplotlib.dates as mdates

In [2]:
#set path
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

In [6]:
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region
#loading GFS and ERA5 data for specific box 

#Choose Forecast model: GFS or ERA5RF
fcst_modelz = ['GFS','ERA5RF']
for fcst_model in fcst_modelz:
    for lat_0,lat_1,lon_0,lon_1 in coordinatez:
        #open anomaly files
        file_anom_p90 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p90.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        file_anom_p10 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p10.nc'.format(lon_0,lon_1-2,lat_0-2,lat_1)
        anom_p90 = xr.open_dataset(path+file_anom_p90).t.squeeze()
        anom_p10 = xr.open_dataset(path+file_anom_p10).t.squeeze()
        #Process data for warm extremes
        #Calculate dates of persitent extremes that are 4 consecutive days or longer
        anom_p90_spec = anom_p90 > 0
        iszero = np.concatenate(([0], np.equal(anom_p90_spec.values, True).view(np.int8), [0]))
        abs_diff = np.abs(np.diff(iszero))
        # Runs start and end where absdiff is 1.
        idxz = np.where(abs_diff == 1)[0].reshape(-1, 2)
        idx_bool = (idxz[:,1]-idxz[:,0])>=4
        hw_idxz = idxz[idx_bool,:]
        hw_dates = np.empty(hw_idxz.shape,dtype=object)
        for i,hw_idx in enumerate(hw_idxz):
            hw_date = [str(anom_p90.time[hw_idx[0]].values),
                       str(anom_p90.time[hw_idx[1]].values-np.timedelta64(1,'D'))]
            hw_dates[i]=hw_date
        np.save(path+'pw_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1),hw_dates)
        np.savetxt(path+'pw_lon_{}_{}_lat_{}_{}.txt'.format(lon_0,lon_1,lat_0,lat_1),hw_dates,fmt="%s")
        #Process data for cold extremes
        anom_p10_spec = anom_p10 < 0
        iszero = np.concatenate(([0], np.equal(anom_p10_spec.values, True).view(np.int8), [0]))
        abs_diff = np.abs(np.diff(iszero))
        # Runs start and end where absdiff is 1.
        idxz = np.where(abs_diff == 1)[0].reshape(-1, 2)
        idx_bool = (idxz[:,1]-idxz[:,0])>=4
        cw_idxz = idxz[idx_bool,:]
        cw_dates = np.empty(cw_idxz.shape,dtype=object)
        for i,cw_idx in enumerate(cw_idxz):
            cw_date = [str(anom_p10.time[cw_idx[0]].values),
                       str(anom_p10.time[cw_idx[1]].values-np.timedelta64(1,'D'))]
            cw_dates[i]=cw_date
        np.save(path+'cw_lon_{}_{}_lat_{}_{}.npy'.format(lon_0,lon_1,lat_0,lat_1),cw_dates)
        np.savetxt(path+'cw_lon_{}_{}_lat_{}_{}.txt'.format(lon_0,lon_1,lat_0,lat_1),cw_dates,fmt="%s")
        